In [1]:
!pip install pandas_ta

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import finplot as fplt

In [4]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import numpy as np

# Getting Data
We utilise the yfinance module to scrape OHLC data for 'company' from dates 01/01/18 to 01/01/21.


Dates 01/02/23 to 28/01/23 will be used for backtesting.

In [5]:
#Company="BB" Period=10 years
#Train: 2014-2021
#Test: 2022-2024
df = yf.download("BB", period="10y")
df["Change"] = df["Close"] - df["Open"]
df["Change %"] = (df["Change"] / df["Open"])*100
train=df.iloc[:2013]
test=df.iloc[2013:]
names=[]
for col in df.columns:
    df.rename(columns={col:col.lower()},inplace=True)
df

[*********************100%%**********************]  1 of 1 completed


,open,high,low,close,adj close,volume,change,change %
Date,,,,,,,,
2014-01-06,7.61,8.03,7.61,8.01,8.01,37202800,0.400000,5.256243
2014-01-07,8.10,8.58,8.10,8.50,8.50,71568800,0.400000,4.938267
2014-01-08,8.84,9.12,8.36,8.54,8.54,52052800,-0.300000,-3.393667
2014-01-09,8.65,8.74,8.42,8.72,8.72,25118600,0.070001,0.809256
2014-01-10,8.92,9.01,8.71,8.76,8.76,24845500,-0.160000,-1.793720
...,...,...,...,...,...,...,...,...
2023-12-28,3.55,3.62,3.51,3.56,3.56,5432700,0.010000,0.281690
2023-12-29,3.55,3.62,3.53,3.54,3.54,7023400,-0.010000,-0.281690
2024-01-02,3.50,3.51,3.39,3.43,3.43,6794900,-0.070000,-1.999998


In [6]:
def get_adx(high, low, close, lookback):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()
    return plus_di, minus_di, adx_smooth

def NormToPi(ARR):
    Arr = np.copy(ARR)
    K1 = 10
    while(Arr[len(Arr)-1]/K1 > 1): K1 *= 10
    LX = Arr[len(Arr)-1]/K1
    K2 = 1
    while(LX * K2 < np.pi): K2 += 0.01
    K2-= 0.01
    return [ (Value/K1)*K2  for Value in Arr]

def Operator(XS, YS, N = 100, Fr = "4.1"):
    Xs = NormToPi(XS)
    Ys = np.copy(YS)

    def F(x, E=0.01):
        while(E < 100):
            Nm = 0
            for Value in Xs:
                if (abs(x - Value) < E):
                    return Ys[Nm]
                Nm += 1
            E *= 10

    def S(k, x, n = 100):
        Sn = (((-1) ** k) * np.sin(n * x)) / (n * x - k * np.pi)
        return Sn

    def Ln(f, x, n = 100):
        LN = 0
        for k in range(1, n):
            Sk = S(k,x,n)
            xi = (k * np.pi) / n
            LN +=  Sk * F(xi)
        return LN

    def Ah(f, x, n = 100):
        AH = 0
        for k in range(1, n):
            AH += (((S(k-1, x, n) + S(k, x, n)) * f(k * (np.pi/n)))/2)
        return AH

    if(Fr == "4.1"):     return [Ln(F, x, n = N) for x in Xs]
    if(Fr == "5.38"):   return [Ah(F, x, n = N) for x in Xs]

In [7]:
df['plus_di'] = pd.DataFrame(get_adx(df['high'], df['low'], df['close'], 14)[0]).rename(columns = {0:'plus_di'})
df['minus_di'] = pd.DataFrame(get_adx(df['high'], df['low'], df['close'], 14)[1]).rename(columns = {0:'minus_di'})
df['adx'] = pd.DataFrame(get_adx(df['high'], df['low'], df['close'], 14)[2]).rename(columns = {0:'adx'})

In [8]:
df[['close','plus_di','minus_di','adx']].plot(figsize=(12,10), color=['black','green','red','orange'])

In [9]:
def HA(df, ohlc=['open', 'high', 'low', 'close']):
    '''
    Function to compute Heiken Ashi Candles (HA)

    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        ohlc: List defining OHLC Column names (default ['open', 'high', 'low', 'close'])

    Returns :
        df : Pandas DataFrame with new columns added for
            Heiken Ashi Close (HA_$ohlc[3])
            Heiken Ashi Open (HA_$ohlc[0])
            Heiken Ashi High (HA_$ohlc[1])
            Heiken Ashi Low (HA_$ohlc[2])'''


    ha_open = 'HA_' + ohlc[0]
    ha_high = 'HA_' + ohlc[1]
    ha_low = 'HA_' + ohlc[2]
    ha_close = 'HA_' + ohlc[3]

    df[ha_close] = (df[ohlc[0]] + df[ohlc[1]] + df[ohlc[2]] + df[ohlc[3]]) / 4

    df[ha_open] = 0.00
    for i in range(0, len(df)):
        if i == 0:
            df[ha_open].iat[i] = (df[ohlc[0]].iat[i] + df[ohlc[3]].iat[i]) / 2
        else:
            df[ha_open].iat[i] = (df[ha_open].iat[i - 1] + df[ha_close].iat[i - 1]) / 2

    df[ha_high]=df[[ha_open, ha_close, ohlc[1]]].max(axis=1)
    df[ha_low]=df[[ha_open, ha_close, ohlc[2]]].min(axis=1)

    return df

In [10]:
df=HA(df)
fplt.candlestick_ochl(df[['HA_open', 'HA_close', 'HA_high', 'HA_low']])
fplt.show()

QPainter::drawPicture: Painter not active


# Creating Heiken-Ashi candle data
This allows us to normalize and smoothen the curves, helping us reduce noise and have a clearer idea of the undelying trend.


In [11]:
'''use ta to get candlestick data'''

'''put graphs and visualize this data'''

'''remember to use adj_close and not close'''

'remember to use adj_close and not close'

In [12]:
df

,open,high,low,close,adj close,volume,change,change %,plus_di,minus_di,adx,HA_close,HA_open,HA_high,HA_low
Date,,,,,,,,,,,,,,,
2014-01-06,7.61,8.03,7.61,8.01,8.01,37202800,0.400000,5.256243,NaN,NaN,NaN,7.8150,7.810000,8.030000,7.610000
2014-01-07,8.10,8.58,8.10,8.50,8.50,71568800,0.400000,4.938267,NaN,NaN,NaN,8.3200,7.812500,8.580000,7.812500
2014-01-08,8.84,9.12,8.36,8.54,8.54,52052800,-0.300000,-3.393667,NaN,NaN,NaN,8.7150,8.066250,9.120000,8.066250
2014-01-09,8.65,8.74,8.42,8.72,8.72,25118600,0.070001,0.809256,NaN,NaN,NaN,8.6325,8.390625,8.740000,8.390625
2014-01-10,8.92,9.01,8.71,8.76,8.76,24845500,-0.160000,-1.793720,NaN,NaN,NaN,8.8500,8.511562,9.010000,8.511562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-28,3.55,3.62,3.51,3.56,3.56,5432700,0.010000,0.281690,14.694328,29.528831,23.150462,3.5600,3.627199,3.627199,3.510000
2023-12-29,3.55,3.62,3.53,3.54,3.54,7023400,-0.010000,-0.281690,14.634237,29.408076,23.892904,3.5600,3.593599,3.620000,3.530000
2024-01-02,3.50,3.51,3.39,3.43,3.43,6794900,-0.070000,-1.999998,14.834587,35.644016,24.621496,3.4575,3.576800,3.576800,3.390000
